In [20]:
# AUTHOR : ABDELHAMID LARACHI
# CVRP AND TSP WITH ANT COLONY AND EXACT METHODS

import math
import random 
from datetime import datetime
import numpy as np
from random import randrange
from itertools import permutations 
import itertools
import time


hour = datetime.now().time().hour # get current hour

trafficByHour = [[0, 0, 10], # traffic by time interval (24h) {index[0]=> hour, index[1]=> min, index[2]=> max}
     [1, 0, 10], 
     [2, 0, 10], 
     [3, 0, 15],
     [4, 0, 15], 
     [5, 5, 20], 
     [6, 10, 30],
     [7, 50, 100], 
     [8, 70, 100], 
     [9, 40, 100],
     [10, 20, 80], 
     [11, 20, 80], 
     [12, 40, 100],
     [13, 40, 100], 
     [14, 10, 50], 
     [15, 30, 80],
     [16, 60, 100], 
     [17, 70, 100], 
     [18, 40, 90],
     [19, 20, 60], 
     [20, 20, 60], 
     [21, 10, 40],
     [22, 10, 20], 
     [23, 10, 20]]

minimum = trafficByHour[hour][1]   # set maximum traffic for the current time interval
maximum = trafficByHour[hour][2]   # set minimum traffic for the current time interval

mSize=10 # matrix size
quart = round(25*mSize/100) # get matrix quarter to generate tsp in vrp range
tspLength = random.randint(4, mSize-quart)  # generate traveling salesman problem length

vrp=[]
nTsp = 3

def generateTSPfromVRP(): #generate multiple travel salesman problems
    tsp=[]
    for i in range(20):  # add random nodes to our tsp in range of our matrix's size (VRP)
        r=random.randint(1,mSize-1)
        if r not in tsp: 
            tsp.append(r)
    vrp.append(tsp)

for i in range(nTsp):
    generateTSPfromVRP() # generate random traveling salesman problem in range of VRP
    
    

matrix = np.random.randint(1, size=(mSize, mSize))  # generate random binary matrix
pheromone = np.full((mSize, mSize), 1.01)  # generate pheromone matrix
temp = pheromone # renistilize pheromone table for another tsp

def generateMatrix():
    i=0
    for head in matrix:
        j=0
        for neighbor in head:
            if neighbor == 0: # if node is a neighbor for the current head
                matrix[j][i] = random.randint(minimum, maximum) # generate random ponderation between traffic interval
            matrix[i][j]=matrix[j][i] # swap/replace vertical and horizental value
            matrix[i][i]=i # set sommet id
            pheromone[i][i]=i
            j=j+1
        i=i+1

generateMatrix()
print('Matrix: \n')
print(matrix, '\n')

# sPath is a function that follow the first looking shortest path from current node each time
def sPath(ants, i, alpha, beta, decay):
    solution = [i] # final route (i = initial node)
    cu = i #current node
    for node in ants: 
        filtre = [] # set of all remaining paths of connected neighbors, to get the shortest one among all
        for n in ants:
            if n!=cu and solution.count(n) == 0: # reject current node index and all visited nodes
                filtre.append(Amatrix[cu][n]) # get all routes to others remaining neighbors

        if len(filtre)>0:
            s = min(filtre) # get shortest path among all neighbors of the current node
            for x in ants:
                row = pheromone[cu][x] ** alpha * (( 1.0 / Amatrix[cu][x]) ** beta)
                if Amatrix[cu][x] == s and solution.count(Amatrix[cu][x]) == 0: # get index of shortest value
                    solution.append(x) 
                    Amatrix[cu][x]=Amatrix[cu][x]*row
                    pheromone[cu][x]=pheromone[cu][x]/decay # spread pheromone
                else: 
                    pheromone[cu][x]=pheromone[cu][x]*decay # evaporate pheromone

        cu = solution[len(solution)-1] # save next index as current index
    solution.append(i) # return to initial node
    return solution

def antColony(ants, n_iterations): #n_ants == number of nodes
    p_solutions=[] # get all ants chosen routes by iteration
    d_avg=[] # get all ponderation and pheromon average in routes chosen by ants by iteration
    iterations=[] # store iteration number of each route (additionel info)
    for i in range(0, n_iterations, 1):
        decay=0.95
        for n in ants:
            solution = sPath(ants, n, alpha=1, beta=1, decay=0.95)
            p_solutions.append(solution)
            t = dCounter(solution)
            d_avg.append(t)
            iterations.append(i)
    indexMin = d_avg.index(min(d_avg))
    shortestPath = p_solutions[indexMin]
    print("\nFinal Pheromone Matrix :\n")
    print(pheromone)
    print("\nFinal Ponderation and Pheromone Avg in Matrix :\n")
    print(Amatrix)
    print("\nBest Solution by Ant Colony : ", shortestPath)
            

# dCounter is a function to calculate ponderation between 2 nodes
def dCounter(solution):
    t = 0
    for i in range(0, len(solution)-1, 1):
        s = solution[i+1] # next node
        p = solution[i] # current node
        d = matrix[p][s] # distance between
        t = t+d
        #print(p, "=>", s, ": ", d)
    return solution, t # return combination and its total ponderation
    


def filtre(): # in order to get accurate results from possible solutions we have to filtre it
    global population
    temp = [] # temporary array to filter possible solutions
    for c in population:
        solution, t = dCounter(c)
        if t==bSolution: # if we use != instead, removing items from population will skip indexes
            temp.append(c) # so it will not check all our list
    population = temp
    print('\nAll possible solutions with same minimum ponderation : ', population)
        
        
# getCombinations is a function for getting all possible combinations from a tsp
def getCombinations(tsp):
    global bSolution
    perm = permutations(tsp) # factoriel tsp to get all combinations
    #print('All combinations :\n')
    for i in perm:
        i += (i[0],) #add initial node to the end of our tuple object (combination)
        solution, t = dCounter(i) # assign returned variables from ponderation counter function (combination, ponderation)
        combinations.append(solution)
        # print(solution, t)
        if t<=bSolution or bSolution==0: # if best solution is not assigned then save first solution as the best one 
            bSolution = t # if solution is <= then keep it as best solution and add it to population
            population.append(solution)
    print('\nMinimum Ponderation: ', bSolution)
    filtre()


def capacityCounter(combination):
    totalCapacity = 0
    for node in range(0, len(combination), 1):
        totalCapacity = totalCapacity + demand[combination[node]]
    return totalCapacity

def getBestFit():
    for c in n_combination:
        j = capacityCounter(c)
        if j <= capacity[0]:
            combinationCapacity.append(j) 
        else:
            combinationCapacity.append(0) # eliminate if bigger than truck capacity but keep its index

    bestFit = min(combinationCapacity, key=lambda x:abs(x-capacity[0])) # get closest value to truck capacity
    bestCombinations = np.where(combinationCapacity == bestFit)[0]
    optimalSolutions = [] 
    for o in bestCombinations:
        optimalSolutions.append(n_combination[o]) # get all combinations for best fit if exist more than one
    
    print('\nDemand by node : ', demand)
    print('\nCapacity by truck : ', capacity)
    print('\n',bestFit, 'is the closest to our truck capacity (',capacity[0],')')
    print('\nBest capacity fit : ', bestFit)
    print('\nAll possible solutions (max capacity fit) (',len(optimalSolutions), 'found ) : ', optimalSolutions)

def shuffleCapacity(): # shuffle all combinations of nodes in order to calculate demand not route
    nodes = []
    for i in range(1, len(matrix)-1, 1):
        nodes.append(i)
    for L in range(0, len(nodes)+1):
        for c in itertools.combinations(nodes, L): 
            #print(c)
            n_combination.append(c)
    n_combination.pop(0) 
    print('\nn_combination (total demand of each combination):', n_combination)
    getBestFit()

print('VRP IS : ', vrp)
for tsp in vrp:
    pheromone = temp # renistilize pheromone table for another tsp
    combinations = []
    combinationCapacity = []
    bSolution = 0 # save as best solution if better than older
    population = [] # other possible solutions with same ponderation
    Amatrix = matrix.astype(np.float) # convert matrix to float for antColony algorithm
    demand = np.random.randint(10,50,(1,len(matrix)))[0] # generate random demand list (index==node)
    demand[0]=0 # set index 0 as depot
    capacity = [100, 100, 100] # trucks capacity (index==truck)
    n_combination = [] # all possible combinations of a groupe of nodes [1,2], [3,1], [3,2]...
    
    print('\nCURRENT TSP IS : ', tsp)
    print('_______________________________________________')
    print('USING ALL POSSIBLE COMBINATIONS -BEST SOLUTION-')
    print('_______________________________________________')
    start_time = time.time()
    getCombinations(tsp)
    print("\nExecution Time :")
    print("--- %s seconds ---" % (time.time() - start_time))
    print('_______________________________________________')
    print('USING ANT COLONY ALGORITHM -GOOD SOLUTION-')
    print('_______________________________________________')
    start_time = time.time()
    antColony(tsp, 1)
    print("\nExecution Time :")
    print("--- %s seconds ---" % (time.time() - start_time))
    print('_______________________________________________')
    print('BEST CAPACITY FIT  -BEST SOLUTION-')
    print('_______________________________________________')
    start_time = time.time()
    shuffleCapacity()
    print("\nExecution Time :")
    print("--- %s seconds ---" % (time.time() - start_time))

Matrix: 

[[ 0 44 15 32 37 40 50 18 14 21]
 [44  1 22 25 31 38 26 19 47 34]
 [15 22  2 47 25 23 15 28 30 45]
 [32 25 47  3 33 21 24 25 24 50]
 [37 31 25 33  4 44 30 23 39 39]
 [40 38 23 21 44  5 32 38 28 23]
 [50 26 15 24 30 32  6 13 41 20]
 [18 19 28 25 23 38 13  7 15 22]
 [14 47 30 24 39 28 41 15  8 12]
 [21 34 45 50 39 23 20 22 12  9]] 

VRP IS :  [[5, 7, 4, 9, 2, 6, 8, 3], [5, 1, 2, 9, 3, 6, 4, 8], [5, 4, 1, 2, 3, 9, 7, 8]]

CURRENT TSP IS :  [5, 7, 4, 9, 2, 6, 8, 3]
_______________________________________________
USING ALL POSSIBLE COMBINATIONS -BEST SOLUTION-
_______________________________________________

Minimum Ponderation:  157

All possible solutions with same minimum ponderation :  [(5, 9, 8, 7, 6, 2, 4, 3, 5), (5, 3, 4, 2, 6, 7, 8, 9, 5), (7, 6, 2, 4, 3, 5, 9, 8, 7), (7, 8, 9, 5, 3, 4, 2, 6, 7), (4, 2, 6, 7, 8, 9, 5, 3, 4), (4, 3, 5, 9, 8, 7, 6, 2, 4), (9, 5, 3, 4, 2, 6, 7, 8, 9), (9, 8, 7, 6, 2, 4, 3, 5, 9), (2, 4, 3, 5, 9, 8, 7, 6, 2), (2, 6, 7, 8, 9, 5, 3, 4, 2), (6, 7


Minimum Ponderation:  166

All possible solutions with same minimum ponderation :  [(5, 3, 1, 2, 4, 7, 8, 9, 5), (5, 9, 8, 7, 4, 2, 1, 3, 5), (4, 2, 1, 3, 5, 9, 8, 7, 4), (4, 7, 8, 9, 5, 3, 1, 2, 4), (1, 2, 4, 7, 8, 9, 5, 3, 1), (1, 3, 5, 9, 8, 7, 4, 2, 1), (2, 4, 7, 8, 9, 5, 3, 1, 2), (2, 1, 3, 5, 9, 8, 7, 4, 2), (3, 5, 9, 8, 7, 4, 2, 1, 3), (3, 1, 2, 4, 7, 8, 9, 5, 3), (9, 5, 3, 1, 2, 4, 7, 8, 9), (9, 8, 7, 4, 2, 1, 3, 5, 9), (7, 4, 2, 1, 3, 5, 9, 8, 7), (7, 8, 9, 5, 3, 1, 2, 4, 7), (8, 9, 5, 3, 1, 2, 4, 7, 8), (8, 7, 4, 2, 1, 3, 5, 9, 8)]

Execution Time :
--- 0.3688480854034424 seconds ---
_______________________________________________
USING ANT COLONY ALGORITHM -GOOD SOLUTION-
_______________________________________________

Final Pheromone Matrix :

[[0.         1.01       1.01       1.01       1.01       1.01
  1.01       1.01       1.01       1.01      ]
 [1.01       0.56880009 1.77566779 0.57448809 0.57448809 0.57448809
  0.82265131 0.70532067 0.57448809 0.57448809]
 [1.01  